# TensorRT

## Установка зависимостей

In [ ]:

!pip list | grep torch

torch                                    2.8.0+cu126
torchao                                  0.10.0
torchaudio                               2.8.0+cu126
torchdata                                0.11.0
torchsummary                             1.5.1
torchtune                                0.6.1
torchvision                              0.23.0+cu126


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.1 MB/s eta 0:00:00
  Created wheel for tensorrt: filename=tensorrt-10.13.3.9-py2.py3-none-any.whl size=46401 sha256=6516fbbe155abf908c28d13cdc45d2cc7f8d817eb551d34fae3e36cefa598950
  Stored in directory: /root/.cache/pip/wheels/1a/aa/b0/561559321c1659e43c5a6661986c3072f9c383efeeaaffb1a5
  Created wheel for tensorrt_cu13: filename=tensorrt_cu13-10.13.3.9-py2.py3-none-any.whl size=17436 sha256=d113087bef5edb4b1f57221970b31fddb8993fa18b0521dd0c3c97d56f387ef9
  Stored in directory: /root/.cache/pip/wheels/df/

  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-pyindex: filename=nvidia_pyindex-1.0.9-py3-none-any.whl size=8419 sha256=672a6ec9f9a30b6dc67f7a62b530736f953280fa6bd6d73be54b68dddbbe8929
  Stored in directory: /root/.cache/pip/wheels/eb/2d/7f/d86cb060a9c51fb933aa4fe0d2f73ffe8df2bd0b58d3d2bba4
Successfully built nvidia-pyindex
  Using cached tensorrt-10.13.3.9-py2.py3-none-any.whl
  Using cached tensorrt_cu13-10.13.3.9-py2.py3-none-any.whl
  Using cached tensorrt_cu13_libs-10.13.3.9-py2.py3-none-manylinux_2_28_x86_64.whl
  Using cached tensorrt_cu13_bindings-10.13.3.9-cp312-none-manylinux_2_28_x86_64.whl.metadata (606 bytes)
  Using cached nvidia_cuda_runtime_cu13-0.0.1.tar.gz (1.4 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Using cached tensorrt_cu13_bindings-10.13.3.9-cp312-none-manylinux_2_28_x86_64.whl (1.2 MB)
  error: subprocess-exited-with-error
  
  × Building wheel 

In [1]:
!pip3 install torch-tensorrt==2.8.0 -f https://github.com/pytorch/TensorRT/releases/expanded_assets/2.8.0
!pip install -U "nvidia-modelopt[all]"

Looking in links: https://github.com/pytorch/TensorRT/releases/expanded_assets/2.8.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 76.0 MB/s eta 0:00:00
  Created wheel for tensorrt: filename=tensorrt-10.12.0.36-py2.py3-none-any.whl size=46638 sha256=3def4fc0c33b906e26797a8902cf6578b25c210a6b77730fcca9e473881d45ed
  Stored in directory: /root/.cache/pip/wheels/0e/d5/2f/cc5e3e56d49c61a02a7a8313f37db27d9af00e7f3463ed33e7
  Created wheel for tensorrt_cu12: filename=tensorrt_cu12-10.12.0.36-py2.py3-none-any.whl size=17480 sha256=708b6ac64be8000eaf601569a4ea55648493c3a05567df3bd5961c9d6553aaf5
  Stored in directo

## Датасет

In [10]:
import torchvision
from torchvision import transforms
import torch
from torch import nn
import torch_tensorrt
import modelopt.torch.quantization as mtq

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

testing_dataset = torchvision.datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ]
    ),
)

testing_dataloader = torch.utils.data.DataLoader(
    testing_dataset, batch_size=1, shuffle=False, num_workers=1
)


### Функция калибровки

In [21]:
def forward_loop(model):
    for img, _ in testing_dataloader:
        img = img.to(device)
        model(img)



### Модель

In [26]:
model = torchvision.models.resnet18().to(device)

### Квантизация при помощи TensorRT Model Optimiser

In [ ]:
from functools import partial

config = mtq.INT8_DEFAULT_CFG

quantized_model = mtq.quantize(model, config, forward_loop)

Inserted 107 quantizers


In [ ]:
mtq.print_quant_summary(quantized_model)

# Задание на семинар

Нужно квантизировать при помощи TensorRT любую модель из torchvision или timm (hugging-face) до int8 и до float16. Затем нужно проверить скорость работы получившихся вариаций модели (float32 - исходная, float16 и int8) и их размер. Результаты привести в блокноте.

In [ ]:
!pip install timm torchao

In [ ]:
from typing import Tuple
from torchao.utils import (
    benchmark_model,
    unwrap_tensor_subclass,
)
from torch import nn
import torch

import os

def benchmark_speed(model_orig: nn.Module,
                    model_quant: nn.Module,
                    example_inputs: torch.Tensor,
                    num_runs: int = 100):
    torch._dynamo.reset()
    orig_time = benchmark_model(model_orig, num_runs, example_inputs)
    quant_time = benchmark_model(model_quant, num_runs, example_inputs)

    print("orig mean time: %0.3f ms" % orig_time)
    print("quant mean time: %0.3f ms" % quant_time)
    print("speedup: %0.1fx" % (orig_time / quant_time))
    torch._dynamo.reset()


def benchmark_size(model_orig: nn.Module, model_quant: nn.Module):
    torch.save(model_orig, "/tmp/orig_model.pt")
    torch.save(model_quant, "/tmp/quant_model.pt")
    quant_model_size_mb = os.path.getsize("/tmp/quant_model.pt") / 1024 / 1024
    orig_model_size_mb = os.path.getsize("/tmp/orig_model.pt") / 1024 / 1024

    print("quant model size: %.2f MB" % quant_model_size_mb)

    print("original model size: %.2f MB" % orig_model_size_mb)


In [ ]:
import timm

model = timm.create_model('vit_base_patch14_dinov2.lvd142m').cuda()
model




VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=Fal

In [ ]:
import torchvision
from torchvision import transforms

testing_dataset = torchvision.datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
            transforms.Resize((518, 518))
        ]
    ),
)

testing_dataloader = torch.utils.data.DataLoader(
    testing_dataset, batch_size=1, shuffle=False, num_workers=1
)

In [ ]:
from torch_tensorrt.ts.ptq import DataLoaderCalibrator, CalibrationAlgo

calibrator = DataLoaderCalibrator(
    testing_dataloader,
    cache_file="./calibration.cache",
    use_cache=False,
    algo_type=CalibrationAlgo.ENTROPY_CALIBRATION_2,
    device=torch.device("cuda:0"),
)


  calibrator = DataLoaderCalibrator(



In [ ]:
quantized_model = torch_tensorrt.compile(model, inputs=[torch_tensorrt.Input((1, 3, 518, 518))],
                                    enabled_precisions={torch.float, torch.half, torch.int8},
                                    # calibrator=calibrator,
                                    device='cuda:0')

ERROR:torch_tensorrt [TensorRT Conversion Context]:IBuilder::buildSerializedNetwork: Error Code 4: Internal Error (Calibration failure occurred with no scaling factors detected. This could be due to no int8 calibrator or insufficient custom scales for network layers. Please see int8 sample to setup calibration correctly.)


AssertionError: 

In [ ]:
# benchmark_speed(model, quantized_model, torch.rand(1, 3, 518, 518).cuda())
benchmark_size(model, quantized_model)

quant model size: 937.36 MB
original model size: 330.39 MB
